# Visualization: Trading Session

In [330]:
import lxml.html as lh
import time
import urllib.request
import argparse
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs

import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns
from IPython import display

### 1. Define parameters and Load model

In [331]:
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)



tz = pytz.timezone('Asia/Kolkata')

path1 = os.getcwd()
path = path1 + '/chromedriver'

ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(executable_path=path , options=options)

model_name = 'model_debug_10'
test_stock = 'data/GOOG_2019.csv'
window_size = 10
debug = True


2020-05-22 12:43:05 MacBook-Pro-2.local selenium.webdriver.remote.remote_connection[6663] DEBUG POST http://127.0.0.1:49350/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}
2020-05-22 12:43:05 MacBook-Pro-2.local urllib3.connectionpool[6663] DEBUG Starting new HTTP connection (1): 127.0.0.1
2020-05-22 12:43:06 MacBook-Pro-2.local urllib3.connectionpool[6663] DEBUG http://127.0.0.1:49350 "POST /session HTTP/1.1" 200 726
2020-05-22 12:43:06 MacBook-Pro-2.local selenium.webdriver.remote.remote_connection[6663] DEBUG Finished Request


### 2. Load test data

In [332]:
#This function returns the table of the given url
def Real(url,count):
    if count == 0:
        
        driver.get(url)
        #print(driver)

    else:
        driver.refresh()
        time.sleep(15)

    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    live = float(live[0].text.replace(',',''))
    return live 

def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
  
    print(t)
        # select an action
    action = agent.act(state, is_eval=True)

        # BUY
    if action == 1:
        agent.inventory.append(data[t])

        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))

        # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta

        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
    else:
        history.append((data[t], "HOLD"))

#        done = (t == data_length - 1)
    agent.memory.append((state, action, reward, next_state))

    return total_profit, history

def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] =  pd.Series(position)
    df['action'] = pd.Series(actions)
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [333]:
def run_live_model_test(ticker):
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = ticker + '.NS'
    price = []
    window_size = 10
    time_now = datetime.datetime.now(tz).time()
    df = pd.DataFrame(columns=['actual', 'date'])
    while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
        url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
        live = Real(url,count)
        count+=1      
        price.append(live)
        df = df.append({'actual': live,
                        'date': datetime.datetime.now(tz).replace(microsecond=0).isoformat()}, ignore_index=True)
        if count < window_size:
           continue
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        test_result, history = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, test_result, initial_offset)
        #plot real-time price action with buy/hold/sell signals from the model
        chart = visualize(df, history, title=test_stock)
        display.clear_output(wait=True)
        display.display(chart)
        t+=1
        state = next_state      

### 3. Running Eval

In [334]:
run_live_model_test('RELIANCE')
#initially, chart appears after 10 requests.
#chart updates every second with latest price
#zoom out to look at hourly/daily data

alt.LayerChart(...)

2020-05-22 12:46:12 MacBook-Pro-2.local selenium.webdriver.remote.remote_connection[6663] DEBUG POST http://127.0.0.1:49350/session/4b776fb56b70ecc926ab5ac4853b65c6/refresh {}
2020-05-22 12:46:13 MacBook-Pro-2.local urllib3.connectionpool[6663] DEBUG http://127.0.0.1:49350 "POST /session/4b776fb56b70ecc926ab5ac4853b65c6/refresh HTTP/1.1" 200 14
2020-05-22 12:46:13 MacBook-Pro-2.local selenium.webdriver.remote.remote_connection[6663] DEBUG Finished Request


KeyboardInterrupt: 